In [54]:
import pandas as pd
import os 


In [55]:
feature_name_df = pd.read_excel("data/1992_Tab_v2/v2.xlsx").loc[8:]
feature_name_df = pd.Series(feature_name_df.values[:,0])


In [56]:
feature_name_df = feature_name_df.str.split(n=1, expand=True) 

feature_name_df[1] = feature_name_df[1].str.strip()

In [57]:
feature_name_df.columns = ['index', 'overflow']

In [58]:
feature_name_df = feature_name_df[['index']].join(feature_name_df["overflow"].str.split(n=1, expand=True) )

In [59]:
feature_name_df.columns = ['index','feature_code', 'feature_name']

In [60]:
feature_name_df = feature_name_df.dropna(subset=["index"], axis=0)

In [61]:
feature_name_df= feature_name_df.reset_index(drop=True)

In [62]:
text_wrapped_df = feature_name_df[~(feature_name_df["index"].str.contains(r'^\d+$')) ]#.index.tolist()
#feature_name_df[~((feature_name_df[1].str.contains("item")) | (feature_name_df[1].str.contains("flag")))]


#text_wrapped_df = feature_name_df[~(feature_name_df["index"].str.contains(r'^\d+$'))]#.index.tolist()

def print_consecutive(lst):
    for i in range(len(lst) - 1):
        if lst[i] + 1 == lst[i+1]:
            print(lst[i], lst[i+1])


print_consecutive(text_wrapped_df.index.tolist())
len(text_wrapped_df)

179

In [63]:
text_wrapped_df

,index,feature_code,feature_name
15,"(dollars),",1992,None
18,"(dollars),",1992,None
21,farm,"(dollars),",1992
50,"(dollars),",1992,None
53,&,greenhouse,"crops ($1,000), 1992"
...,...,...,...
5698,"farms),",1992,None
5705,"(farms),",1992,None
5708,(land,in,"farms), 1992"
5758,in,"farms),",1992


In [64]:
text_wrapped_df = text_wrapped_df.fillna('')

text_wrapped_df = text_wrapped_df['index']+ ' ' + text_wrapped_df['feature_code']+ ' ' + text_wrapped_df['feature_name']

In [65]:
text_wrapped_df.index = text_wrapped_df.index -1

In [66]:
text_wrapped_df.name = "next_line_overflow"

In [67]:
feature_name_df = feature_name_df.join(text_wrapped_df)

In [68]:
feature_name_df["feature_name"] = feature_name_df["feature_name"]+feature_name_df["next_line_overflow"].fillna("")

In [69]:
idx_to_drop = ( text_wrapped_df.index +1).tolist()

In [70]:
feature_name_df = feature_name_df.drop(idx_to_drop)

In [71]:
pd.set_option('display.max_colwidth', None)
feature_name_df

,index,feature_code,feature_name,next_line_overflow
0,1,stateicp,ICPSR state code,NaN
1,2,counicp,ICPSR county code,NaN
2,3,name,Name of area,NaN
3,4,fips,State/county FIPS code,NaN
4,5,statefip,State FIPS code,NaN
...,...,...,...,...
5787,5565,item390010,"Grazing permits, Source of permits (Taylor grazing), 1992",NaN
5788,5566,flag390011,Flag for item390011,NaN
5789,5567,item390011,"Grazing permits, Source of permits (Indian land), 1992",NaN
5790,5568,flag390012,Flag for item390012,NaN


In [72]:
feature_name_df.reset_index(inplace=True,drop=True)

In [73]:
feature_name_df.drop("next_line_overflow",axis=1, inplace=True)

In [74]:
text_wrapped_df = feature_name_df[~(feature_name_df["index"].str.contains(r'^\d+$'))]#.index.tolist()


In [75]:
feature_name_df["index"] = feature_name_df["index"].astype(int)

In [76]:
feature_name_df["index"] 

0          1
1          2
2          3
3          4
4          5
        ... 
5608    5565
5609    5566
5610    5567
5611    5568
5612    5569
Name: index, Length: 5613, dtype: int64

In [77]:
def non_consecutive_items(lst):
    non_consecutive = []
    for i in range(len(lst) - 1):
        if lst[i] + 1 != lst[i+1]:
            non_consecutive.append(lst[i])
    # Since the last item doesn't have a subsequent one to compare with, 
    # we always add it to the non_consecutive list.
    non_consecutive.append(lst[-1])
    return non_consecutive

#non_consecutive_items(feature_name_df["index"].tolist( ))

In [78]:
df_1992_ = feature_name_df[feature_name_df["index"]==1992].drop(2020)

feature_name_df.loc[df_1992_.index-1, "feature_name"]  += " 1992"

In [79]:
feature_name_df = feature_name_df.drop(df_1992_.index).reset_index(drop=True)

In [80]:
feature_name_df["feature_name"] = feature_name_df["feature_name"].str.strip()

In [81]:
feature_name_df["feature_name"].replace(r'\s{2,}', ' ', regex=True, inplace=True)


In [82]:
fc_save_path = os.path.join(data_folder,'processed','feature_codebook_1992.csv')

In [83]:
feature_name_df.to_csv(fc_save_path, index=False)

In [84]:
feature_df

,state,county,name,level,statefip,counfip,fips,flag010001,item010001,flag010002,...,flag390008,item390008,flag390009,item390009,flag390010,item390010,flag390011,item390011,flag390012,item390012
0,100,0,UNITED STATES,3,0,0,0,0,1925300,0,...,0,10498,0,9232,0,12927,0,2000,0,11831
1,41,0,ALABAMA,2,1,0,1000,0,37905,0,...,0,0,0,15,0,11,0,10,0,60
2,41,10,AUTAUGA,1,1,1,1001,0,322,0,...,5,0,5,0,5,0,5,0,5,0
3,41,30,BALDWIN,1,1,3,1003,0,941,0,...,5,0,5,0,5,0,5,0,5,0
4,41,50,BARBOUR,1,1,5,1005,0,421,0,...,5,0,5,0,5,0,5,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174,68,390,TETON,1,56,39,56039,0,102,0,...,0,7,0,23,0,2,0,0,0,5
3175,68,410,UINTA,1,56,41,56041,0,265,0,...,0,30,0,18,0,64,0,0,0,9
3176,68,430,WASHAKIE,1,56,43,56043,0,207,0,...,0,35,0,15,0,70,0,0,0,16
3177,68,450,WESTON,1,56,45,56045,0,231,0,...,0,78,0,73,0,38,0,2,0,24


In [86]:
~feature_df.columns.str.contains('flag')

array([ True,  True,  True, ...,  True, False,  True])

In [87]:
cols_to_drop = [col for col in feature_df.columns if "flag" in col.lower()]
feature_df.drop(columns=cols_to_drop)

,state,county,name,level,statefip,counfip,fips,item010001,item010002,item010003,...,item390003,item390004,item390005,item390006,item390007,item390008,item390009,item390010,item390011,item390012
0,100,0,UNITED STATES,3,0,0,0,1925300,945531520,491,...,5133,4056,3186,3670,3701,10498,9232,12927,2000,11831
1,41,0,ALABAMA,2,1,0,1000,37905,8450823,223,...,56,16,3,3,3,0,15,11,10,60
2,41,10,AUTAUGA,1,1,1,1001,322,107259,333,...,0,0,0,0,0,0,0,0,0,0
3,41,30,BALDWIN,1,1,3,1003,941,167832,178,...,0,0,0,0,0,0,0,0,0,0
4,41,50,BARBOUR,1,1,5,1005,421,177189,421,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3174,68,390,TETON,1,56,39,56039,102,62307,611,...,1,7,4,3,6,7,23,2,0,5
3175,68,410,UINTA,1,56,41,56041,265,879694,3320,...,1,2,7,12,17,30,18,64,0,9
3176,68,430,WASHAKIE,1,56,43,56043,207,397883,1922,...,8,6,7,9,9,35,15,70,0,16
3177,68,450,WESTON,1,56,45,56045,231,1484583,6427,...,1,1,2,3,12,78,73,38,2,24
